## Import

In [2]:
import os
import random
import pickle
import gc
import warnings
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.preprocessing import (
    StandardScaler, PowerTransformer, OrdinalEncoder,
    OneHotEncoder, FunctionTransformer, PolynomialFeatures, LabelEncoder
)
from category_encoders import TargetEncoder, BinaryEncoder
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import (
    LogisticRegression, LinearRegression, Ridge, Lasso,
    SGDRegressor, ElasticNet
)
from sklearn.model_selection import (
    train_test_split, cross_val_score, cross_validate,
    GridSearchCV, KFold, cross_val_predict
)
from sklearn.metrics import (
    roc_auc_score, mean_squared_error, make_scorer, accuracy_score
)
from sklearn import set_config, datasets
from catboost import (
    CatBoostRegressor, CatBoostClassifier
)
from sklearn.pipeline import (
    Pipeline, FeatureUnion, make_pipeline
)
from sklearn.ensemble import (
    RandomForestClassifier, StackingClassifier, StackingRegressor,
    GradientBoostingRegressor, VotingClassifier, VotingRegressor,
    HistGradientBoostingRegressor, GradientBoostingClassifier,
    BaggingClassifier, AdaBoostClassifier, RandomForestRegressor
)
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.svm import SVC, SVR, LinearSVC
from xgboost import XGBRegressor, XGBClassifier 
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_moons

import xgboost as xgb
import lightgbm as lgb
import category_encoders as ce
import re
import optuna
import math

%matplotlib inline

warnings.filterwarnings("ignore")

## Fixed Random-Seed

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

## Load Data

In [4]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
trade_df = pd.read_csv('international_trade.csv')

item: 품목 코드  
TG : 감귤  
BC : 브로콜리  
RD : 무  
CR : 당근  
CB : 양배추

계절, 시즌, 

In [5]:
train_df

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0
...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0


## Data Pre-Processing

In [6]:
#시계열 특성을 학습에 반영하기 위해 timestamp를 월, 일, 시간으로 나눕니다
train_df['year'] = train_df['timestamp'].apply(lambda x : int(x[0:4]))
train_df['month'] = train_df['timestamp'].apply(lambda x : int(x[5:7]))
train_df['day'] = train_df['timestamp'].apply(lambda x : int(x[8:10]))

test_df['year'] = test_df['timestamp'].apply(lambda x : int(x[0:4]))
test_df['month'] = test_df['timestamp'].apply(lambda x : int(x[5:7]))
test_df['day'] = test_df['timestamp'].apply(lambda x : int(x[8:10]))

trade_df['year'] = trade_df['기간'].apply(lambda x : int(x[0:4]))
trade_df['month'] = trade_df['기간'].apply(lambda x : int(x[5:7]))

In [7]:
#학습에 사용하지 않을 변수들을 제거합니다
train_x = train_df.drop(columns=['ID', 'timestamp', 'supply(kg)', 'price(원/kg)']) #supply를 빼는 게 맞을까
train_y = train_df['price(원/kg)']

test_x = test_df.drop(columns=['ID', 'timestamp'])
trade = trade_df.drop(columns=['기간'])
trade = trade.rename(columns = {'품목명' : 'item'})

In [8]:
display(train_x,trade)

,item,corporation,location,year,month,day
0,TG,A,J,2019,1,1
1,TG,A,J,2019,1,2
2,TG,A,J,2019,1,3
3,TG,A,J,2019,1,4
4,TG,A,J,2019,1,5
...,...,...,...,...,...,...
59392,RD,F,J,2023,2,27
59393,RD,F,J,2023,2,28
59394,RD,F,J,2023,3,1
59395,RD,F,J,2023,3,2


,item,수출 중량,수출 금액,수입 중량,수입 금액,무역수지,year,month
0,토마토(신선한 것이나 냉장한 것으로 한정한다),356571,990,0,0,990,2019,1
1,양파,821330,222,4003206,1118,-896,2019,1
2,쪽파,60,1,93405,128,-127,2019,1
3,꽃양배추와 브로콜리(broccoli),160,1,638913,563,-562,2019,1
4,방울다다기 양배추,0,0,7580,38,-38,2019,1
...,...,...,...,...,...,...,...,...
1269,포포(papaw)[파파야(papaya)],0,0,23830,71,-71,2023,2
1270,사과,135165,351,0,0,351,2023,2
1271,배,2206012,5411,1,0,5411,2023,2
1272,신 체리[프루너스 체라서스(Prunus cerasus)],5,0,0,0,0,2023,2


In [9]:
trade['item'] = trade['item'].replace({'감귤': 'TG', '꽃양배추와 브로콜리(broccoli)': 'BC', '당근': 'CR', '양배추': 'CB'})

In [10]:
filtered_df = trade[trade['item'].str.contains('TG|BC|RD|CR|CB')]

### 여기서 무 관련 데이터를 만들어야 됨  
### 그나마 양배추가 가장 수확시기가 비슷해 보임

In [11]:
filtered_df

,item,수출 중량,수출 금액,수입 중량,수입 금액,무역수지,year,month
3,BC,160,1,638913,563,-562,2019,1
5,CB,184650,94,395802,90,4,2019,1
8,CR,23150,22,7466150,2955,-2934,2019,1
17,TG,58368,172,0,0,172,2019,1
28,BC,780,1,396870,399,-398,2019,2
...,...,...,...,...,...,...,...,...
1236,TG,81509,269,0,0,269,2023,1
1248,BC,24,0,332640,352,-352,2023,2
1250,CB,13188,13,377456,104,-91,2023,2
1253,CR,22510,20,9260020,3758,-3737,2023,2


In [12]:
merged_data = pd.merge(train_x, trade, on=['item', 'year', 'month'], how='left')

In [13]:
merged_data

,item,corporation,location,year,month,day,수출 중량,수출 금액,수입 중량,수입 금액,무역수지
0,TG,A,J,2019,1,1,58368.0,172.0,0.0,0.0,172.0
1,TG,A,J,2019,1,2,58368.0,172.0,0.0,0.0,172.0
2,TG,A,J,2019,1,3,58368.0,172.0,0.0,0.0,172.0
3,TG,A,J,2019,1,4,58368.0,172.0,0.0,0.0,172.0
4,TG,A,J,2019,1,5,58368.0,172.0,0.0,0.0,172.0
...,...,...,...,...,...,...,...,...,...,...,...
59392,RD,F,J,2023,2,27,NaN,NaN,NaN,NaN,NaN
59393,RD,F,J,2023,2,28,NaN,NaN,NaN,NaN,NaN
59394,RD,F,J,2023,3,1,NaN,NaN,NaN,NaN,NaN
59395,RD,F,J,2023,3,2,NaN,NaN,NaN,NaN,NaN


In [14]:
merged_data['수출 중량(나눔)'] = merged_data['수출 중량'] / 30
merged_data['수출 금액(나눔)'] = merged_data['수출 금액'] / 30
merged_data['수입 중량(나눔)'] = merged_data['수입 중량'] / 30
merged_data['수입 금액(나눔)'] = merged_data['수입 금액'] / 30
merged_data['무역수지(나눔)'] = merged_data['무역수지'] / 30

merged_data = merged_data.drop(columns= ['수출 중량', '수출 금액', '수입 중량','수입 금액','무역수지'])

### corporation이 F인 경우 index가 달라지는 상황  
### trade에서 값을 변형하고 merge 하는 게 낫다고 판단함

In [15]:
merged_data[merged_data['item'] == 'TG']

,item,corporation,location,year,month,day,수출 중량(나눔),수출 금액(나눔),수입 중량(나눔),수입 금액(나눔),무역수지(나눔)
0,TG,A,J,2019,1,1,1945.600000,5.733333,0.0,0.000000,5.733333
1,TG,A,J,2019,1,2,1945.600000,5.733333,0.0,0.000000,5.733333
2,TG,A,J,2019,1,3,1945.600000,5.733333,0.0,0.000000,5.733333
3,TG,A,J,2019,1,4,1945.600000,5.733333,0.0,0.000000,5.733333
4,TG,A,J,2019,1,5,1945.600000,5.733333,0.0,0.000000,5.733333
...,...,...,...,...,...,...,...,...,...,...,...
15225,TG,E,S,2023,2,27,229.833333,1.133333,925.5,3.266667,-2.133333
15226,TG,E,S,2023,2,28,229.833333,1.133333,925.5,3.266667,-2.133333
15227,TG,E,S,2023,3,1,NaN,NaN,NaN,NaN,NaN
15228,TG,E,S,2023,3,2,NaN,NaN,NaN,NaN,NaN


In [16]:
merged_data[merged_data['item'] == 'CR']

,item,corporation,location,year,month,day,수출 중량(나눔),수출 금액(나눔),수입 중량(나눔),수입 금액(나눔),무역수지(나눔)
15230,CR,A,J,2019,1,1,771.666667,0.733333,248871.666667,98.500000,-97.800000
15231,CR,A,J,2019,1,2,771.666667,0.733333,248871.666667,98.500000,-97.800000
15232,CR,A,J,2019,1,3,771.666667,0.733333,248871.666667,98.500000,-97.800000
15233,CR,A,J,2019,1,4,771.666667,0.733333,248871.666667,98.500000,-97.800000
15234,CR,A,J,2019,1,5,771.666667,0.733333,248871.666667,98.500000,-97.800000
...,...,...,...,...,...,...,...,...,...,...,...
25886,CR,E,S,2023,2,27,750.333333,0.666667,308667.333333,125.266667,-124.566667
25887,CR,E,S,2023,2,28,750.333333,0.666667,308667.333333,125.266667,-124.566667
25888,CR,E,S,2023,3,1,NaN,NaN,NaN,NaN,NaN
25889,CR,E,S,2023,3,2,NaN,NaN,NaN,NaN,NaN


In [17]:
merged_data[merged_data['item'] == 'CB']

,item,corporation,location,year,month,day,수출 중량(나눔),수출 금액(나눔),수입 중량(나눔),수입 금액(나눔),무역수지(나눔)
25891,CB,A,J,2019,1,1,6155.0,3.133333,13193.400000,3.000000,0.133333
25892,CB,A,J,2019,1,2,6155.0,3.133333,13193.400000,3.000000,0.133333
25893,CB,A,J,2019,1,3,6155.0,3.133333,13193.400000,3.000000,0.133333
25894,CB,A,J,2019,1,4,6155.0,3.133333,13193.400000,3.000000,0.133333
25895,CB,A,J,2019,1,5,6155.0,3.133333,13193.400000,3.000000,0.133333
...,...,...,...,...,...,...,...,...,...,...,...
57869,CB,F,J,2023,2,27,439.6,0.433333,12581.866667,3.466667,-3.033333
57870,CB,F,J,2023,2,28,439.6,0.433333,12581.866667,3.466667,-3.033333
57871,CB,F,J,2023,3,1,NaN,NaN,NaN,NaN,NaN
57872,CB,F,J,2023,3,2,NaN,NaN,NaN,NaN,NaN


In [18]:
merged_data[merged_data['item'] == 'RD']

,item,corporation,location,year,month,day,수출 중량(나눔),수출 금액(나눔),수입 중량(나눔),수입 금액(나눔),무역수지(나눔)
31983,RD,A,J,2019,1,1,NaN,NaN,NaN,NaN,NaN
31984,RD,A,J,2019,1,2,NaN,NaN,NaN,NaN,NaN
31985,RD,A,J,2019,1,3,NaN,NaN,NaN,NaN,NaN
31986,RD,A,J,2019,1,4,NaN,NaN,NaN,NaN,NaN
31987,RD,A,J,2019,1,5,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
59392,RD,F,J,2023,2,27,NaN,NaN,NaN,NaN,NaN
59393,RD,F,J,2023,2,28,NaN,NaN,NaN,NaN,NaN
59394,RD,F,J,2023,3,1,NaN,NaN,NaN,NaN,NaN
59395,RD,F,J,2023,3,2,NaN,NaN,NaN,NaN,NaN


In [19]:
merged_data[merged_data['item'] == 'BC']

,item,corporation,location,year,month,day,수출 중량(나눔),수출 금액(나눔),수입 중량(나눔),수입 금액(나눔),무역수지(나눔)
42644,BC,A,J,2019,1,1,5.333333,0.033333,21297.1,18.766667,-18.733333
42645,BC,A,J,2019,1,2,5.333333,0.033333,21297.1,18.766667,-18.733333
42646,BC,A,J,2019,1,3,5.333333,0.033333,21297.1,18.766667,-18.733333
42647,BC,A,J,2019,1,4,5.333333,0.033333,21297.1,18.766667,-18.733333
42648,BC,A,J,2019,1,5,5.333333,0.033333,21297.1,18.766667,-18.733333
...,...,...,...,...,...,...,...,...,...,...,...
56346,BC,E,S,2023,2,27,0.800000,0.000000,11088.0,11.733333,-11.733333
56347,BC,E,S,2023,2,28,0.800000,0.000000,11088.0,11.733333,-11.733333
56348,BC,E,S,2023,3,1,NaN,NaN,NaN,NaN,NaN
56349,BC,E,S,2023,3,2,NaN,NaN,NaN,NaN,NaN


In [20]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59397 entries, 0 to 59396
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   item         59397 non-null  object 
 1   corporation  59397 non-null  object 
 2   location     59397 non-null  object 
 3   year         59397 non-null  int64  
 4   month        59397 non-null  int64  
 5   day          59397 non-null  int64  
 6   수출 중량(나눔)    47120 non-null  float64
 7   수출 금액(나눔)    47120 non-null  float64
 8   수입 중량(나눔)    47120 non-null  float64
 9   수입 금액(나눔)    47120 non-null  float64
 10  무역수지(나눔)     47120 non-null  float64
dtypes: float64(5), int64(3), object(3)
memory usage: 5.0+ MB


### 무에 대한 결측치를 어떻게 처리를 하는 게 좋을지

In [21]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')

# 결측치가 있는 열에 대해 fit_transform을 사용하여 평균값으로 대체합니다.
merged_data[['수출 중량(나눔)', '수출 금액(나눔)', '수입 중량(나눔)', '수입 금액(나눔)', '무역수지(나눔)']] = imputer.fit_transform(merged_data[['수출 중량(나눔)', '수출 금액(나눔)', '수입 중량(나눔)', '수입 금액(나눔)', '무역수지(나눔)']])


In [22]:
#질적 변수들을 수치화합니다
qual_col = ['item', 'corporation', 'location']

for i in qual_col:
    le = LabelEncoder()
    train_x[i]=le.fit_transform(train_x[i])
    test_x[i]=le.transform(test_x[i]) #test 데이터에 대해서 fit하는 것은 data leakage에 해당합니다

print('Done.')

Done.


## Regression Model Fit

In [23]:
## 앙상블 모델 정의

cat = CatBoostRegressor(random_state = 2024, 
                            n_estimators = 1000, 
                            learning_rate = 0.01, 
                            depth = 10,
                            l2_leaf_reg = 3,
                            metric_period = 1000)

xgb = XGBRegressor(n_estimators = 1000, random_state = 2024, learning_rate = 0.01, max_depth = 10)


# voting
vote_model = VotingRegressor(
    estimators =[("cat",cat), ("xgb", xgb)]
)

vote_model.fit(train_x,train_y)

pred = vote_model.predict(test_x)
for idx in range(len(pred)):
    if pred[idx]<0:
        pred[idx]= 0

0:	learn: 2018.2432359	total: 153ms	remaining: 2m 33s
999:	learn: 1162.7545461	total: 7.21s	remaining: 0us


In [24]:
# model = RandomForestRegressor()
# model.fit(train_x, train_y)

## Inference

In [25]:
# preds = model.predict(test_x)

## Submission

In [26]:
submission1 = pd.read_csv('./sample_submission.csv')

In [27]:
submission1['answer'] = pred

In [28]:
submission1.to_csv('./baseline_submission.csv', index=False)